In [26]:
import pandas as pd

### Download the data and load it to Pandas. 

You can find the data files in the README.

In [30]:
# titles = pd.read_csv('data/titles.csv')
titles = pd.read_csv('titles.csv')
# titles.head()
titles.head()

,title,year
0,The Rising Son,1990
1,The Thousand Plane Raid,1969
2,Crucea de piatra,1993
3,Country,2000
4,Gaiking II,2011


In [31]:
# cast = pd.read_csv('data/cast.csv')
cast = pd.read_csv('cast.csv')
# cast.head()
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


## Part 1: Group By

### 1. Using groupby(), count the number of films that have been released in each decade in the history of cinema.

In [9]:
# Extract the decade from the release year
titles['decade'] = (titles['year'] // 10) * 10

# Group by decade and count the number of films
films_per_decade = titles.groupby('decade')['title'].count()

print(films_per_decade)

decade
1890        3
1900       37
1910     6512
1920     8797
1930    10097
1940     8576
1950    12711
1960    17515
1970    18714
1980    20243
1990    22225
2000    38713
2010    67976
2020      210
Name: title, dtype: int64


### 2. Use groupby() count the number of "Hamlet" films made in each decade.

In [13]:
# Filter for films with the title 'Hamlet'
hamlet_films = titles[titles['title'] == 'Hamlet']

# Extract the decade from the release year
hamlet_films['decade'] = (hamlet_films['year'] // 10) * 10

# Group by decade and count the number of "Hamlet" films
films_per_decade = hamlet_films.groupby('decade')['title'].count()

print(films_per_decade)

decade
1910    3
1920    1
1940    1
1950    1
1960    2
1970    2
1980    1
1990    2
2000    2
2010    3
Name: title, dtype: int64


C:\Users\a7902\AppData\Local\Temp\ipykernel_22516\906543853.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hamlet_films['decade'] = (hamlet_films['year'] // 10) * 10


### 3. Use groupby() to determine how many roles are listed for each of the Pink Panther movies.

In [38]:

# Filter the titles DataFrame to include only the Pink Panther movies
pink_panther_titles = titles[titles['title'].str.contains('Pink Panther', case=False, na=False)]

# Merge the filtered titles with the cast DataFrame to get the roles
pink_panther_cast = cast[cast['title'].isin(pink_panther_titles['title'])]

# Group by movie title and count the number of roles
role_counts = pink_panther_cast.groupby('title').size().reset_index(name='role_count')

# Print the result
print(role_counts)

                            title  role_count
0       Curse of the Pink Panther          67
1     Revenge of the Pink Panther          64
2         Son of the Pink Panther          46
3                The Pink Panther         111
4              The Pink Panther 2          84
5  The Pink Panther Strikes Again          68
6  The Return of the Pink Panther          30
7       Trail of the Pink Panther          41


### 4. Using groupby(), list each of the films in which Frank Oz has played more than 1 role in order by year.

In [39]:
# Filter the DataFrame to include only rows where Frank Oz is listed
frank_oz_cast = cast[cast['name'] == 'Frank Oz']

# Group by title and year and count the number of roles
role_counts = frank_oz_cast.groupby(['title', 'year']).size().reset_index(name='role_count')

# Filter to include only those films where the count of roles is greater than 1
frank_oz_multiple_roles = role_counts[role_counts['role_count'] > 1]

# Sort the resulting DataFrame by year
frank_oz_multiple_roles_sorted = frank_oz_multiple_roles.sort_values(by='year')

# Print the result
print(frank_oz_multiple_roles_sorted)

                                   title  year  role_count
22                      The Muppet Movie  1979           8
0         An American Werewolf in London  1981           2
20                The Great Muppet Caper  1981           6
19                      The Dark Crystal  1982           2
23            The Muppets Take Manhattan  1984           7
2                       Follow That Bird  1985           3
21            The Muppet Christmas Carol  1992           7
7                 Muppet Treasure Island  1996           4
8                     Muppets from Space  1999           4
17  The Adventures of Elmo in Grouchland  1999           3


## Part 2: Merging

For these questions, you'll also need the following dataframe on release dates.

You'll need to merge to join together the information present in different dataframes, in addition to other filtering and calculations.

In [40]:
# release_dates = pd.read_csv('data/release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)
release_dates = pd.read_csv('release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)
# release_dates.head()
release_dates.head()

C:\Users\a7902\AppData\Local\Temp\ipykernel_22516\1742529517.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  release_dates = pd.read_csv('release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)


,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### 1. Count the number of movies with "Christmas" in their title for each month, that is released are the USA.

In [44]:
# Filter out rows where the 'title' is NaN
titles = titles.dropna(subset=['title'])

# Filter the titles DataFrame to include only movies with "Christmas" in their title
christmas_titles = titles[titles['title'].str.contains('Christmas', case=False, na=False)]

# Merge the Christmas titles with the release_dates DataFrame to get the release dates
christmas_release_dates = christmas_titles.merge(release_dates, on='title')

# Filter to include only movies released in the USA
usa_christmas_releases = christmas_release_dates[christmas_release_dates['country'] == 'USA']

# Convert release date to datetime format
usa_christmas_releases['release_date'] = pd.to_datetime(usa_christmas_releases['date'], errors='coerce')

# Drop rows where 'release_date' is NaN after conversion
usa_christmas_releases = usa_christmas_releases.dropna(subset=['release_date'])

# Extract the month from the release date
usa_christmas_releases['month'] = usa_christmas_releases['release_date'].dt.month

# Group by month and count the number of movies
month_counts = usa_christmas_releases.groupby('month').size().reset_index(name='movie_count')

# Print the result
print(month_counts)


    month  movie_count
0       1            2
1       2            3
2       4            1
3       5            2
4       6            2
5       7            1
6       8            2
7       9            2
8      10           16
9      11           47
10     12           56


C:\Users\a7902\AppData\Local\Temp\ipykernel_22516\2002717172.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_christmas_releases['release_date'] = pd.to_datetime(usa_christmas_releases['date'], errors='coerce')


### 2. Count the number of movies with "Romance" in their title for **each day of the week**, that are released in the USA.

In [43]:
# Filter out rows where the 'title' is NaN
titles = titles.dropna(subset=['title'])

# Filter the titles DataFrame to include only movies with "Christmas" in their title
christmas_titles = titles[titles['title'].str.contains('Christmas', case=False, na=False)]

# Merge the Christmas titles with the release_dates DataFrame to get the release dates
christmas_release_dates = christmas_titles.merge(release_dates, on=['title', 'year'])

# Filter to include only movies released in the USA
usa_christmas_releases = christmas_release_dates[christmas_release_dates['country'] == 'USA']

# Convert release date to datetime format
usa_christmas_releases['release_date'] = pd.to_datetime(usa_christmas_releases['date'], errors='coerce')

# Drop rows where 'release_date' is NaN after conversion
usa_christmas_releases = usa_christmas_releases.dropna(subset=['release_date'])

# Extract the month from the release date
usa_christmas_releases['month'] = usa_christmas_releases['release_date'].dt.month

# Group by month and count the number of movies
month_counts = usa_christmas_releases.groupby('month').size().reset_index(name='movie_count')

# Print the result
print(month_counts)

    month  movie_count
0       1            2
1       2            2
2       4            1
3       5            1
4       6            1
5       7            1
6       8            2
7       9            2
8      10           14
9      11           38
10     12           47


C:\Users\a7902\AppData\Local\Temp\ipykernel_22516\775123346.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_christmas_releases['release_date'] = pd.to_datetime(usa_christmas_releases['date'], errors='coerce')


### 3. On which date was each Judi Dench movie from the 1990s released in the USA?

In [46]:
# Filter out rows where the 'title' or 'year' is NaN
titles = titles.dropna(subset=['title', 'year'])

# Filter the titles DataFrame to include only movies where Judi Dench is listed in the cast
judi_dench_movies = titles[titles['title'].str.contains('Judi Dench', case=False, na=False)]

# Filter further to include only movies released in the 1990s
judi_dench_1990s = judi_dench_movies[(judi_dench_movies['year'] >= 1990) & (judi_dench_movies['year'] <= 1999)]

# Merge the Judi Dench 1990s movies with the release_dates DataFrame to get the release dates
judi_dench_release_dates = judi_dench_1990s.merge(release_dates, on=['title', 'year'])

# Filter to include only movies released in the USA
usa_judi_dench_releases = judi_dench_release_dates[judi_dench_release_dates['country'] == 'USA']

# Select relevant columns to display
result = usa_judi_dench_releases[['title', 'date']]  # Assuming 'date' is the column name in release_dates.csv for release date

# Print the result
print(result)

Empty DataFrame
Columns: [title, date]
Index: []


### 4. In which months do films with the actress Judi Dench tend to be released in the USA?